In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb
import HEI
import sys

# Dictionaries and lists

In [2]:
important=['Participant ID','VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
       'VEG0600','VEG0900','VEG0500','VEG0100','VEG0700','FRU0100','FRU0200','FRU0300','FRU0400',
       'FRU0500','FRU0600','FRU0700','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
       'Whole Grains (ounce equivalents)','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
       'DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
       'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500',
       'DOT0600','DOT0100','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
       'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0100','MFL0100',
       'MFF0200','MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400',
       'MOF0500','MOF0600','MOF0700','VEG0700','MFF0100','MFL0100','MFF0200','MSL0100','MSF0100',
       'MOF0500','MOF0600','MOF0700','VEG0700','Sodium (mg)','Refined Grains (ounce equivalents)',
       'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
       'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
       'Total Saturated Fatty Acids (SFA) (g)','FRU0100','FRU0200','DMF0200','DMR0200','DML0200','DML0300',
       'SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600','SWT0100',
       'SWT0200','SWT0300','SWT0700' , 'SWT0800','SWT0400','SWT0500','DOT0300' , 'DOT0400',
       'DOT0100','DOT0200','GRR0800' , 'GRS0800',  'GRW0800','GRW0900' ,'GRS0900' , 'GRR0900' ,
       'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400','VEG0800','FMC0100', 'FMC0200','DMF0100',
       'DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600','DOT0700',
       'DOT0800','GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700','GRW1300','GRS1300']

para_dict = {
        'hei_totveg': {'parameters':[1.1], 'name': 'HEIX1_TOTALVEG'},
        'hei_greensbeans': {'parameters':[0.2], 'name': 'HEIX2_GREEN_AND_BEAN'},
        'hei_totfruit': {'parameters':[0.8], 'name': 'HEIX3_TOTALFRUIT'},
        'hei_wholefruit': {'parameters':[0.4], 'name': 'HEIX4_WHOLEFRUIT'},
        'hei_wholegrains': {'parameters':[1.5], 'name': 'HEIX5_WHOLEGRAIN'},
        'hei_dairy': {'parameters':[1.3], 'name': 'HEIX6_TOTALDAIRY'},
        'hei_totproteins': {'parameters':[2.5], 'name': 'HEIX7_TOTPROT'},
        'hei_seafoodplantprot': {'parameters':[0.8], 'name': 'HEIX8_SEAPLANT_PROT'},
        'hei_refinedgrains': {'parameters':[1.8,4.3], 'name': 'HEIX11_REFINEDGRAIN'},
        'hei_addedsugars': {'parameters':[6.5,26], 'name': 'HEIX12_ADDEDSUGARS'},
        'hei_SFA': {'parameters':[8,16], 'name': 'HEIX13_SATFATS'},
        'Fats': {'parameters':[1.2,2.5], 'name': 'HEIX9_FATTYACID'},
        'hei_sodium':{'parameters':[1.1,2.0],'name':'HEIX10_SODIUM'}
        }

ped811_dict = {
    'hei_vegetables': {'parameters':[0.1,1.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,1.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.0,3.5,0.0 , 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[20,28,8,35], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.5,6.0,0,10.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.6,3.5], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
        }

ped1224_dict ={
    'hei_vegetables': {'parameters':[0.1,7.9], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0.1,7.9], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[1.5, 5.5, 0, 8.0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_milk': {'parameters':[14.0,18.0,8.0,24.0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[2.0,3.0,0,6.0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[1.9, 4.2], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[4.1,6.0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0.1,4.0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0.1,1.0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0.1,1.0], 'name': 'HEIX10_SALTY'}
        }

ped08_dict = {
    'hei_vegetables': {'parameters':[0,0], 'name': 'HEIX1_VEGETABLES'},
    'hei_totfruit': {'parameters':[0,0], 'name': 'HEIX2_TOTALFRUIT'},
    'hei_wholegrains': {'parameters':[0,0,0,0], 'name': 'HEIX3_WHOLEGRAIN'},
    'hei_dairy': {'parameters':[0,0,0,0], 'name': 'HEIX4_TOTALDAIRY'},
    'hei_proteins': {'parameters':[0,0,0,0], 'name': 'HEIX5_PROTEIN'},
    'hei_refinedgrains': {'parameters':[0,0], 'name': 'HEIX6_REFINEDGRAIN'},
    'hei_fruitjuice': {'parameters':[0,0], 'name': 'HEIX7_FRUITJUICE'},
    'hei_SSB': {'parameters':[0,0], 'name': 'HEIX8_SSB'},
    'hei_sweets': {'parameters':[0,0], 'name': 'HEIX9_SWEETS'},
    'hei_salty': {'parameters':[0,0], 'name': 'HEIX10_SALTY'},
    'hei_cereal': {'parameters':[0,0], 'name': 'HEIX11_CEREAL'}
    }

ped_dict={
    'young': ped811_dict,
    'child':ped1224_dict,
    'infant':ped08_dict
}


hei_dict={
      'hei_totveg':
      ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
      'hei_greensbeans':
      ['VEG0100','VEG0700'],
      'hei_totfruit':
      ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
      'hei_wholefruit':
      ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
      'hei_wholegrains':
      ['Whole Grains (ounce equivalents)'],
      'hei_dairy':
      ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                   'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                   'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                   'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
      'hei_totproteins':
      ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                         'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                         'MFF0100','MFL0100','MFF0200','MSL0100',
                         'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                         'MOF0600','MOF0700','VEG0700'],
      'hei_seafoodplantprot':
      ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
      'hei_sodium':
      ['Sodium (mg)'],
      'hei_refinedgrains':
      ['Refined Grains (ounce equivalents)'],
      'hei_addedsugars':
      ['Added Sugars (by Total Sugars) (g)'],
      'ripctsfa': ['% Calories from SFA','Energy (kcal)'],
     'energy':
     ['Energy (kcal)'],
     'fats':
     ['Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
     'Total Saturated Fatty Acids (SFA) (g)']
     }

hei_ped_dict={'hei_totveg':
        ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
        'hei_greensbeans':
        ['VEG0100','VEG0700'],
        'hei_totfruit':
        ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholefruit':
        ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
        'hei_wholegrains':
        ['Whole Grains (ounce equivalents)'],
        'hei_dairy':
        ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                     'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                     'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                     'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
        'hei_totproteins':
        ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                           'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                           'MFF0100','MFL0100','MFF0200','MSL0100',
                           'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                           'MOF0600','MOF0700','VEG0700'],
        'hei_seafoodplantprot':
        ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
        'hei_refinedgrains':
        ['Refined Grains (ounce equivalents)'],
       'energy':
       ['Energy (kcal)'],
        'hei_fruitjuice':
        ['FRU0100','FRU0200'],
        'hei_SSB':
        ['DMF0200','DMR0200','DML0200','DML0300','SWT0600','BVS0400','BVS0300','BVS0500','BVS0100','BVS0200','BVS0600','BVS0700', 'SWT0600'],
        'chocolate_candies':
        ['SWT0100'],
        'candies':
        ['SWT0200'],
        'frosting':
        ['SWT0300'],
        'sweet_sauce':
        ['SWT0700' , 'SWT0800' ],
        'sugar':
        ['SWT0400' ],
        'syrups':
        ['SWT0500' ],
        'Pudding':
        ['DOT0300' , 'DOT0400'],
        'icecream':
        ['DOT0100' ],
        'nondairy_treat':
        ['DOT0200' ],
        'baked_good':
        ['GRR0800' , 'GRS0800',  'GRW0800' ],
        'chips':
        ['GRW0900' ,'GRS0900' , 'GRR0900' , 'GRW1100' , 'GRW1200', 'GRW0400' ,'GRS0400' ,'GRR0400' ],
        'fries':
        ['VEG0800' ],
        'other_fried':
        ['FMC0100', 'FMC0200' ],
        'milk':
        ['DMF0100','DMR0100','DML0100','DMN0100' ,'DMF0200','DMR0200','DML0200','DOT0500', 'DOT0600'],
        'formula_foz':
        ['DOT0700', 'DOT0800'],
        'cereal_oz':
        ['GRW0600','GRS0600','GRR0600','GRW0700','GRS0700','GRR0700'],
        'bbcereal_hcup':
        ['GRW1300','GRS1300']
        }

conv_dict={
    'hei_fruitjuice':4,
    'hei_SSB':8,
    'chocolate_candies':40,
    'candies':15,
    'frosting':35,
    'sugar':4,
    'syrups':.25,
    'Pudding':1,
    'icecream':.5,
    'nondairy_treat':85,
    'baked_good':55,
    'fries': 70,
    'formula_foz':5,
    'bbcereal_hcup':.5
}

make_hei_dict={
    'hei_sweets' :
    ['chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good'],
    'hei_salty' :
    ['chips','other_fried','fries'],
    'hei_vegetables':
    ['hei_totveg', 'hei_greensbeans'],
    'hei_proteins':
    ['hei_totproteins','hei_seafoodplantprot'],
    'hei_milk':
    ['hei_dairy','formula_foz'],
    'hei_cereal':
    ['cereal_oz', 'bbcereal_hcup']
}

interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot', 'Total Polyunsaturated Fatty Acids (PUFA) (g)',
        'Total Monounsaturated Fatty Acids (MUFA) (g)', 'Total Saturated Fatty Acids (SFA) (g)',
        'hei_sodium', 'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','% Calories from SFA']

ped_interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains','hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot',
        'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','chocolate_candies','candies','frosting','sweet_sauce','sugar','syrups','Pudding', 'icecream','nondairy_treat','baked_good',
        'chips','other_fried','fries']


In [3]:
basepath='/Users/gracer/Google Drive/BCP/data'

In [4]:
arglist={'BASEPATH': basepath, 
         'CHILD': True, 
         'NAMES': 'testies', 
         'OPTS': ['Chil'], 
         'SAVE':os.path.join(basepath, 'derivived'),
         'XTRA':'total_dataset.csv'}

In [5]:
for (dirpath, dirnames, filenames) in os.walk(basepath):
        for filename in filenames:
            if filename.endswith('.zip'):
                tmppath=os.sep.join([dirpath, filename])
                with ZipFile(tmppath, 'r') as zipObj:
                   # Get a list of all archived file names from the zip
                   listOfFileNames = zipObj.namelist()
                   # Iterate over the file names
                   for fileName in listOfFileNames:
                       # Check filename endswith txt
                        if fileName.endswith('04.txt'):
                            zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                        if fileName.endswith('09.txt'):
                            zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')

In [6]:
a=HEI.file_org(infile, arglist, important)

this is the item Chil
Chil
Chil
Chil
Final dataframe size:  (931, 226)


In [7]:
df=a[arglist['OPTS'][0]]

In [8]:
df.shape

(659, 382)

In [18]:
def file_org(infile, arglist, important):
    # will create a dictionary with the file paths to all the data
    if arglist['OPTS'] == False:
        arglist['OPTS']=[arglist['NAMES']]
        file_dict = {"set_04": {"%s"%arglist['NAMES']:{}}, "set_09": {"%s"%arglist['NAMES']: {}}}
        file_dict['set_04']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "%s"%arglist['NAMES'] in x]
        file_dict['set_09']["%s"%arglist['NAMES']]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "%s"%arglist['NAMES'] in x]
        arglist['OPTS']=[arglist['NAMES']]

    else:
        file_dict={"set_04":{},"set_09":{}}
        for item in arglist['OPTS']:
            print('this is the item %s'%item)
            file_dict["set_04"][item]= {}
            file_dict["set_09"][item]= {}

        for key,value in file_dict.items():
            keys=list(file_dict[key].keys())
            for k in keys:
                print(k)
                file_dict['set_04']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if '%s'%k in x]
                file_dict['set_09']["%s"%k]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if '%s'%k in x]

    return(file_dict)

In [19]:
a=file_org(infile, arglist, important)

this is the item Chil
Chil
Chil


In [20]:
a

{'set_04': {'Chil': {'files': ['/Users/gracer/Google Drive/BCP/data/temp_txt/UNCChildJuly04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCChild04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChilMar1804.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/ChildOct1804.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildMSTR04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly04.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP04.txt']}},
 'set_09': {'Chil': {'files': ['/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildAP09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChildJuly09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/BCPChildMSTR09.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/ChildOct1809.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNChilMar1809.txt',
    '/Users/gracer/Google Drive/BCP/data/temp_txt/UMNUNCChild09.txt',
    '/Users/gracer/Google Driv

In [22]:
def file_reader(arglist, file_dict):
    for ki in arglist['OPTS']:
        temp_list = []
        print(ki)
        for file in file_dict["set_04"]["%s"%ki]["files"]:
            temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
            temp_df=temp_df.drop([0])
            for val in temp_df["Participant ID"]:
                _id = str(val).lstrip("0").split("_")[0]
                temp_df.replace(val, _id, inplace=True)
            temp_list.append(temp_df)
        dfm4_original = pd.concat(temp_list, ignore_index=True)
        print("Final dataframe size: ", dfm4_original.shape)
        dfm4_original = dfm4_original.sort_values(by="Participant ID")

        for col in dfm4_original:
            if dfm4_original[col].dtype == np.object_:
                dfm4_original[col] = (dfm4_original[col].replace(',','.', regex=True))

        temp_list = []
        for file in file_dict["set_09"]["%s"%ki]["files"]:
            temp_df = pd.read_csv(file,encoding='latin1', sep="\t")
            temp_df=temp_df.drop([0])
            for val in temp_df["Participant ID"]:
                _id = str(val).strip("0").strip(".").split("_")[0]
                temp_df.replace(val, _id, inplace=True)
            temp_list.append(temp_df)
        dfm9_original = pd.concat(temp_list, ignore_index=True)
        dfm9_original = dfm9_original.sort_values(by="Participant ID")

        concat_filepath = os.path.join(arglist['SAVE'],'%s_dataset4.csv'%(ki))
        dfm4_original.to_csv(concat_filepath, index=False, sep=",", header=True)

        concat_filepath = os.path.join(arglist['SAVE'],'%s_dataset9.csv'%(ki))
        dfm9_original.to_csv(concat_filepath, index=False, sep=",", header=True)
    return(dfm9_original, dfm4_original)


In [25]:
b,c=file_reader(arglist, a)

Chil
Final dataframe size:  (931, 226)


In [27]:
c

,Project Abbreviation,Participant ID,Date of Intake,Project Name,Record Type,Participant Name,Gender,Date of Birth,Date of Entry,Day of Intake,...,Data Field 1 Descriptor,Data Field 1 Response,Data Field 2 Descriptor,Data Field 2 Response,Data Field 3 Descriptor,Data Field 3 Response,Data Field 4 Descriptor,Data Field 4 Response,Data Field 5 Descriptor,Data Field 5 Response
89,UMNUNCChild,105040,09/18/2018,UMN + UNC Child,1,NaN,3,NaN,01/10/2019,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
670,UMNChildJuly,105040,06/08/2017,UMN Child Redo July 2018,1,NaN,3,NaN,01/24/2018,4,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
391,BCPChildMSTR,105040,09/07/2017,Baby ConnectoMe Project. CHILD MASTER. Q4 2017,1,NaN,3,NaN,01/24/2018,4,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
390,BCPChildMSTR,105040,06/08/2017,Baby ConnectoMe Project. CHILD MASTER. Q4 2017,1,NaN,3,NaN,01/24/2018,4,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
389,BCPChildMSTR,105040,03/01/2017,Baby ConnectoMe Project. CHILD MASTER. Q4 2017,1,NaN,3,NaN,01/24/2018,3,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
671,UMNChildJuly,105040,09/07/2017,UMN Child Redo July 2018,1,NaN,3,NaN,01/24/2018,4,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
393,BCPChildMSTR,106436,07/26/2017,Baby ConnectoMe Project. CHILD MASTER. Q4 2017,1,NaN,3,NaN,11/09/2017,3,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
392,BCPChildMSTR,106436,07/21/2017,Baby ConnectoMe Project. CHILD MASTER. Q4 2017,1,NaN,3,NaN,12/05/2017,5,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
8,UNCChildJuly,106436,07/21/2017,UNC Child Records. July 2018,1,NaN,3,NaN,12/05/2017,5,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
9,UNCChildJuly,106436,07/26/2017,UNC Child Records. July 2018,1,NaN,3,NaN,11/09/2017,3,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN


In [30]:
mer = pd.merge(b,c, on=['Participant ID','Date of Intake','Project Abbreviation'])

In [31]:
mer

,Project Abbreviation,Participant ID,Date of Intake,FRU0100,FRU0200,FRU0300,FRU0400,FRU0500,FRU0600,FRU0700,...,Data Field 1 Descriptor,Data Field 1 Response,Data Field 2 Descriptor,Data Field 2 Response,Data Field 3 Descriptor,Data Field 3 Response,Data Field 4 Descriptor,Data Field 4 Response,Data Field 5 Descriptor,Data Field 5 Response
0,UMNChildJuly,105040,09/07/2017,0,0,0,2.175,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
1,UMNUNCChild,105040,09/18/2018,0,0,0,2.661,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
2,BCPChildMSTR,105040,03/01/2017,0,0,0,0.079,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
3,BCPChildMSTR,105040,06/08/2017,0.006,0,0,0.961,0.172,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
4,BCPChildMSTR,105040,09/07/2017,0,0,0,2.175,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
5,UMNChildJuly,105040,06/08/2017,0.006,0,0,0.961,0.172,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
6,BCPChildMSTR,106436,07/21/2017,0,0,0,0,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
7,BCPChildMSTR,106436,07/26/2017,0,0,0,0,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
8,UNCChildJuly,106436,07/26/2017,0,0,0,0,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
9,UNCChildJuly,106436,07/21/2017,0,0,0,0,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN


In [38]:
mer.drop_duplicates(subset=['Participant ID', 'Date of Intake'], inplace=True)

In [39]:
mer

,Project Abbreviation,Participant ID,Date of Intake,FRU0100,FRU0200,FRU0300,FRU0400,FRU0500,FRU0600,FRU0700,...,Data Field 1 Descriptor,Data Field 1 Response,Data Field 2 Descriptor,Data Field 2 Response,Data Field 3 Descriptor,Data Field 3 Response,Data Field 4 Descriptor,Data Field 4 Response,Data Field 5 Descriptor,Data Field 5 Response
0,UMNChildJuly,105040,09/07/2017,0,0,0,2.175,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
1,UMNUNCChild,105040,09/18/2018,0,0,0,2.661,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
2,BCPChildMSTR,105040,03/01/2017,0,0,0,0.079,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
3,BCPChildMSTR,105040,06/08/2017,0.006,0,0,0.961,0.172,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
6,BCPChildMSTR,106436,07/21/2017,0,0,0,0,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
7,BCPChildMSTR,106436,07/26/2017,0,0,0,0,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
10,BCPChildMSTR,107008,04/22/2017,0,0,0,0.125,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
11,ChildOct18,107008,04/16/2018,0.004,0,0,5.122,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
12,ChildOct18,107008,02/05/2018,0,0,0,2.500,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
13,ChildOct18,107008,04/15/2018,0.006,0,0,0.822,0,0,0,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN


In [42]:
mer.shape

(643, 394)

In [40]:
total_df=mer.dropna(axis=1, how='all')

In [41]:
total_df

,Project Abbreviation,Participant ID,Date of Intake,FRU0100,FRU0200,FRU0300,FRU0400,FRU0500,FRU0600,FRU0700,...,Whole Grains (ounce equivalents),Refined Grains (ounce equivalents),PUFA 18:3 n-3 (alpha-linolenic acid [ALA]) (g),Solid Fats (g),Gluten (g),Data Field 1 Descriptor,Data Field 2 Descriptor,Data Field 3 Descriptor,Data Field 4 Descriptor,Data Field 5 Descriptor
0,UMNChildJuly,105040,09/07/2017,0,0,0,2.175,0,0,0,...,0.340,0.761,0.634,11.765,1.165,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
1,UMNUNCChild,105040,09/18/2018,0,0,0,2.661,0,0,0,...,1.353,1.540,0.387,20.504,2.260,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
2,BCPChildMSTR,105040,03/01/2017,0,0,0,0.079,0,0,0,...,0.000,0.000,0.005,0.000,0.000,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
3,BCPChildMSTR,105040,06/08/2017,0.006,0,0,0.961,0.172,0,0,...,0.250,0.000,0.069,2.880,0.000,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
6,BCPChildMSTR,106436,07/21/2017,0,0,0,0,0,0,0,...,0.000,0.000,0.004,0.000,0.000,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
7,BCPChildMSTR,106436,07/26/2017,0,0,0,0,0,0,0,...,0.000,0.000,0.004,0.000,0.000,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
10,BCPChildMSTR,107008,04/22/2017,0,0,0,0.125,0,0,0,...,0.000,0.375,0.219,2.456,0.000,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
11,ChildOct18,107008,04/16/2018,0.004,0,0,5.122,0,0,0,...,0.306,2.779,1.755,15.987,3.482,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
12,ChildOct18,107008,02/05/2018,0,0,0,2.500,0,0,0,...,0.919,1.581,0.857,2.003,3.631,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5
13,ChildOct18,107008,04/15/2018,0.006,0,0,0.822,0,0,0,...,0.306,0.753,1.366,6.676,1.013,Data Field 1,Data Field 2,Data Field 3,Data Field 4,Data Field 5


In [43]:
total_df.shape

(643, 382)

In [44]:
complete=total_df[total_df.columns.intersection(important)].dropna()

In [47]:
complete.columns

Index(['% Calories from SFA', 'Added Sugars (by Total Sugars) (g)', 'BVS0100',
       'BVS0200', 'BVS0300', 'BVS0400', 'BVS0500', 'BVS0600', 'BVS0700',
       'DCF0100',
       ...
       'VEG0500', 'VEG0600', 'VEG0700', 'VEG0700', 'VEG0700', 'VEG0700',
       'VEG0800', 'VEG0800', 'VEG0900', 'Whole Grains (ounce equivalents)'],
      dtype='object', length=156)

In [51]:
def diet_maker(dfm9_original,dfm4_original, ki):
    mer = pd.merge(b,c, on=['Participant ID','Date of Intake','Project Abbreviation'])
    mer.drop_duplicates(subset=['Participant ID', 'Date of Intake'], inplace=True)
    
    total_df=mer.dropna(axis=1, how='all')
    complete_df=total_df[total_df.columns.intersection(important)].dropna()

    concat_filepath = os.path.join(arglist['SAVE'],'%s_BCP_datasetTOTAL.csv'%(ki))
    total_df.to_csv(concat_filepath, index=False, sep=",", header=True)

    concat_filepath = os.path.join(arglist['SAVE'],'%s_BCP_datasetINTEREST.csv'%(ki))
    complete_df.to_csv(concat_filepath, index=False, sep=",", header=True)

    return(complete_df)

In [52]:
e = diet_maker(b,c, 'test')

In [54]:
e.shape

(643, 156)

In [9]:
def path_finder(arglist):
    #read in the data
    df1=os.path.join(arglist['BASEPATH'],arglist['SAVE'],arglist['XTRA'])
    print(df1)
    demo_df=pd.read_csv(df1, sep=',',encoding='latin1')
    return(demo_df)

In [10]:
def refactor(infant_df):
    infant_df['breastfed']=infant_df['breastfed'].replace({'no': 0, 'yes': 1})
    infant_df['any_formula']=infant_df['any_formula'].replace({'no': 0, 'yes': 1,'NaN':'NA'})
    infant_df['regular_formula']=infant_df['regular_formula'].replace({'no': 0, 'yes': 1,'NaN':'NA','not_answered':'NA'})
    infant_df['age_fed_dropdown']=infant_df['age_fed_dropdown'].replace({'no': 0, 'yes': 1,'NaN':'NA','not_answered':'NA','never_not_yet':0})
    infant_df['age_stop_dropdown']=infant_df['age_stop_dropdown'].replace({'no': 0, 'yes': 1,'NaN':'NA','not_answered':'NA','never_not_yet':0})
    return(infant_df)

In [11]:
def ager(x):
    # Get the age in months of each recall
    x['DoB'] = pd.to_datetime(x['DoB'])
    x['Date of Intake'] = pd.to_datetime(x['Date of Intake'])
    x['Date_taken'] = pd.to_datetime(x['Date_taken'])
    x['mullen_Date_taken']=pd.to_datetime(x['mullen_Date,taken'])
    
    x['Age_at_Date_of_Intake'] = x['Date of Intake'] - x['DoB']
    x['Age_at_Date_of_Anthros'] = x['Date_taken'] - x['DoB']
    
    x['Age_at_Date_of_Intake']= x['Age_at_Date_of_Intake']/ np.timedelta64(1, 'M')
    x['Age_at_Date_of_Anthros']= x['Age_at_Date_of_Anthros']/ np.timedelta64(1, 'M')
    x['mullen_Date_taken']= x['mullen_Date_taken']/ np.timedelta64(1, 'M')
    return(x)

In [12]:
def BCP(diet_df, arglist):
    diet_df=df
    demo_df = path_finder(arglist)
    y='Participant ID'
    z='Date of Intake'
    diet_df[y]=diet_df[y].convert_objects(convert_numeric=True)
    demo_df[y]=demo_df[y].convert_objects(convert_numeric=True)
    unique1 = list(set(demo_df[y])-set(diet_df[y])) 
    print(unique1)
    unique2 = list(set(diet_df[y])-set(demo_df[y])) 
    print(unique2)
    all_data=demo_df.merge(diet_df, on=[y,z])
    all_data=ager(all_data)
    all_data['Identifiers_visit']=all_data.Identifiers.str.split('x').str[-1].str.split('m').str[0]

    return(all_data)

In [13]:
x=BCP(df, arglist)

/Users/gracer/Google Drive/BCP/data/derivived/total_dataset.csv
[986114, 794626, 947206, 999947, 968715, 826398, 714783, 682023, 218663, 493610, 298032, 62519, 138813, 841279, 262208, 495169, 787525, 505926, 146505, 738379, 986700, 710741, 680534, 871514, 683099, 850013, 705639, 730727, 410729, 863856, 124529, 747645, 476291, 713347, 820360, 704148, 890518, 302746, 505499, 866461, 251549, 824479, 222881, 569511, 640680, 945323, 387247, 951984, 294064, 125105, 810675, 529587, 861367, 232633, 764612, 562893, 633038, 716498, 427732, 200919, 474330, 375518, 272097, 945890, 509669, 960758, 456442, 914176, 277762, 261379, 655107, 657158, 147207, 100619, 931085, 780046, 340751, 613137, 364820, 437532, 223008, 862498, 536876, 523054, 963376, 243511, 866106, 469822, 662338, 904006, 995663, 376673, 736609, 488291, 878949, 709477, 351080, 575848, 309615, 759162, 387450, 887679, 987008, 115072, 89476, 229768, 534927, 888211, 557468, 295330, 236452, 943541, 912823, 672698, 969150, 318409, 826321, 4

KeyError: 'Date of Intake'

In [44]:
x['Identifiers_visit']=x['Identifiers_visit'].astype(np.float16)

In [68]:
x['Age_at_Date_of_Intake']=x['Age_at_Date_of_Intake'].astype(np.float16)
# x['child_feeding_practice-any_formula']=x['child_feeding_practice-any_formula'].astype(np.str)

In [81]:
x['child_feeding_practice-breastfed']=x['child_feeding_practice-breastfed'].astype('category')

In [82]:
x['child_feeding_practice-breastfed'].value_counts()

nan    10146
yes     4012
no         8
Name: child_feeding_practice-breastfed, dtype: int64

In [83]:
y=HEI.make_ped_components(hei_ped_dict, x, conv_dict)

hei_totveg
hei_greensbeans
hei_totfruit
hei_wholefruit
hei_wholegrains
hei_dairy
hei_totproteins
hei_seafoodplantprot
hei_refinedgrains
energy
hei_fruitjuice
hei_SSB
chocolate_candies
candies
frosting
sweet_sauce
sugar
syrups
Pudding
icecream
nondairy_treat
baked_good
chips
fries
other_fried
milk
formula_foz
cereal_oz
bbcereal_hcup


In [84]:
que=HEI.make_hei(y, make_hei_dict)


START
{'hei_sweets': ['chocolate_candies', 'candies', 'frosting', 'sweet_sauce', 'sugar', 'syrups', 'Pudding', 'icecream', 'nondairy_treat', 'baked_good'], 'hei_salty': ['chips', 'other_fried', 'fries'], 'hei_vegetables': ['hei_totveg', 'hei_greensbeans'], 'hei_proteins': ['hei_totproteins', 'hei_seafoodplantprot'], 'hei_milk': ['hei_dairy', 'formula_foz'], 'hei_cereal': ['cereal_oz', 'bbcereal_hcup']}


In [85]:
def splitter(DF):
    DF_child=DF.query('Age_at_Date_of_Intake >= 12 & Identifiers_visit >= 12 & Age_at_Date_of_Anthros >= 12')
    DF_young=DF.query('Age_at_Date_of_Intake < 12 and Age_at_Date_of_Intake >= 8 & Identifiers_visit < 12 and Identifiers_visit >= 8 & Age_at_Date_of_Anthros < 12 and Age_at_Date_of_Anthros >= 8')
    DF_infant=DF.query('Age_at_Date_of_Intake < 8 & Identifiers_visit < 8 & Age_at_Date_of_Anthros < 8')
    df={'DF_child':DF_child, 'DF_young':DF_young, 'DF_infant':DF_infant}
    return(df)

In [86]:
split_dict=splitter(que)

In [87]:
split_dict['DF_young']['Age_at_Date_of_Intake'].describe()

count    237.000000
mean      10.046875
std        1.239258
min        8.210938
25%        8.867188
50%        9.757812
75%       11.101562
max       11.992188
Name: Age_at_Date_of_Intake, dtype: float64

In [88]:
split_dict['DF_infant']['child_feeding_practice-any_formula'].value_counts()

nan             326
not_answered     20
yes              18
Name: child_feeding_practice-any_formula, dtype: int64

In [90]:
split_dict['DF_infant'].shape

(364, 483)

In [92]:
split_dict['DF_infant']['child_feeding_practice-any_formula'].value_counts()

nan             326
not_answered     20
yes              18
Name: child_feeding_practice-any_formula, dtype: int64

In [95]:
split_dict['DF_infant']['child_feeding_practice-any_formula']=split_dict['DF_infant']['child_feeding_practice-any_formula'].astype('category')
split_dict['DF_infant']['child_feeding_practice-breastfed']=split_dict['DF_infant']['child_feeding_practice-breastfed'].astype('category')

In [124]:
def DQI_BF(df, output, age_group):
    if age_group == 'infant':
        df[output]=[15 if row['child_feeding_practice-breastfed'] == 'yes' and row['child_feeding_practice-any_formula'] != 'yes' else 10 if row['child_feeding_practice-breastfed'] == 'yes' and row['child_feeding_practice-any_formula'] == 'yes'else 5 if row['child_feeding_practice-breastfed'] == 'no' else 'NA' for index, row in df.iterrows()]
    else:
        df[output]=[10 if row['child_feeding_practice-age_stop_dropdown'] != 'yes' and row['child_feeding_practice-breastfed'] == 'yes' else 0 for index, row in df.iterrows()]
        
        
        
        

In [125]:
DQI_BF(split_dict['DF_infant'], 'HEIX0_BREASTFEEDING', 'infant')

In [126]:
split_dict['DF_infant']['HEIX0_BREASTFEEDING'].value_counts()

NA    326
15     20
10     18
Name: HEIX0_BREASTFEEDING, dtype: int64

In [120]:
DQI_BF(split_dict['DF_child'], 'HEIX0_BREASTFEEDING', 'child')

In [121]:
split_dict['DF_child']['HEIX0_BREASTFEEDING'].value_counts()

0     2203
10     307
Name: HEIX0_BREASTFEEDING, dtype: int64

In [122]:
DQI_BF(split_dict['DF_young'], 'HEIX0_BREASTFEEDING', 'young')

In [123]:
split_dict['DF_young']['HEIX0_BREASTFEEDING'].value_counts()

0     189
10     48
Name: HEIX0_BREASTFEEDING, dtype: int64

## Important Feeding stuff
* DoB
* Started Any Formula (Months old) NA - No Formula
* Started Regular (in months)
* Ended Formula (months old) NA - No Formula
* Started Solids (months old)
* Ended breastmilk (months old)
* Feeding Practice (First 4 months)
* child_feeding_practice-age_any_formula
* child_feeding_practice-age_fed
* child_feeding_practice-age_fed_dropdown
* child_feeding_practice-age_fed_status
* child_feeding_practice-age_regular_formula
* child_feeding_practice-age_regular_formula_status
* child_feeding_practice-age_stop
* child_feeding_practice-age_stop_dropdown
* child_feeding_practice-age_stop_formula
* child_feeding_practice-age_stop_formula_dropdown
* child_feeding_practice-age_stop_formula_status
* child_feeding_practice-age_stop_status
* child_feeding_practice-any_formula
* child_feeding_practice-breastfed
* child_feeding_practice-regular_formula
* Date_taken
* Date of Intake

In [159]:
def DQI(df, inputt, output, parameter):
    if inputt in ['hei_salty','hei_sweets','hei_SSB','hei_fruitjuice']:
        print('now calculating %s'%output)
        temp=df[inputt]
        MIN=parameter[0]
        MAX=parameter[1]
        df[output]=[2.5 if MIN < x <= MAX else 0 if x > MAX else 5 for x in temp]
    # No limit
    elif inputt in ['hei_vegetables', 'hei_totfruit']:
        print('now calculating %s'%output)
        temp=df[inputt]
        MIN=parameter[0]
        MAX=parameter[1]
        df[output]=[2.5 if MIN < x <= MAX else 0 if x == MIN else 5 for x in temp]
    # Upper limit
    elif inputt in ['hei_milk','hei_proteins']:
        print('now calculating %s'%output)
        temp=df[inputt]
        FARMIN = parameter[0]
        FARMAX = parameter[1]
        MIN = parameter[2]
        MAX = parameter[3]
        df[output]=[5 if MIN < x <= MAX else 2.5 if MIN < x <= FARMIN or MAX < x <=FARMAX else 0 for x in temp]
    elif inputt in ['hei_wholegrains']:
        print('now calculating %s'%output)
        temp=df[inputt]
        FARMIN = parameter[0]
        FARMAX = parameter[1]
        MIN = parameter[2]
        MAX = parameter[3]
        df[output]=[2.5 if MIN < x <= MAX else 1.5 if MIN < x <= FARMIN or MAX < x <=FARMAX else 0 for x in temp]
    elif inputt in ['hei_refinedgrains']:
        print('now calculating %s'%output)
        temp=df[inputt]
        MIN=parameter[0]
        MAX=parameter[1]
        df[output]=[1.5 if MIN < x <= MAX else 0 if x > MAX else 2.5 for x in temp]
    return(df)

In [160]:
def infant_DQI(df, inputt, output, parameter):
    #inputt, output, parameter
    MIN=0
    if inputt in ['hei_salty','hei_sweets','hei_SSB','hei_fruitjuice','hei_refinedgrains','hei_vegetables', 'hei_totfruit',
    'hei_wholegrains','hei_dairy','hei_proteins', 'hei_cereal']:
        print('now calculating infant %s'%output)
        temp=df[inputt]
        df[output]=[5 if x == MIN else 0 for x in temp]
    return(df)

In [185]:
def check(dic, data, name, option, arglist):
    if arglist['CHILD'] == False:
        df=data
        toSum=['HEIX1_TOTALVEG','HEIX2_GREEN_AND_BEAN' , 'HEIX3_TOTALFRUIT' , 'HEIX4_WHOLEFRUIT' ,
               'HEIX5_WHOLEGRAIN' , 'HEIX6_TOTALDAIRY' , 'HEIX7_TOTPROT' , 'HEIX8_SEAPLANT_PROT' , 'HEIX9_FATTYACID' ,
               'HEIX10_SODIUM' , 'HEIX11_REFINEDGRAIN' , 'HEIX12_ADDEDSUGARS' , 'HEIX13_SATFATS']
        for key,values in dic.items():
            print('Calculating score for %s'%key)
            if key in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot',
                        'hei_wholegrains', 'hei_dairy','Fats']:
                if key == 'Fats':
                    adeq_check(df, key, values['name'], values['parameters'])
                else:
                    adeq_check(df, key, values['name'], values['parameters'][0])
            if key in ['hei_sodium','hei_refinedgrains','hei_addedsugars','hei_SFA']:
                mod_check(df, key, values['name'], values['parameters'])
        df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
        concat_filepath = os.path.join(arglist['SAVE'],'%s_%s_HEI.csv'%(option, name))
        df.to_csv(concat_filepath, index=False, sep=",", header=True)
    else:
        df=data
        for key,values in dic.items():
            if key in ['hei_vegetables','hei_totfruit','hei_wholegrains','hei_dairy','hei_milk','hei_proteins','hei_refinedgrains',
            'hei_fruitjuice','hei_SSB','hei_sweets','hei_salty','hei_cereal']:
                print('Calculating score for %s'%key)
                if name != 'infant':
                    toSum=['HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEIX2_TOTALFRUIT' , 'HEIX3_WHOLEGRAIN' , 'HEIX4_TOTALDAIRY' ,
                           'HEIX5_PROTEIN' , 'HEIX6_REFINEDGRAIN' , 'HEIX7_FRUITJUICE' , 'HEIX8_SSB', 'HEIX9_SWEETS',
                           'HEIX10_SALTY']
                    DQI(df,key, values['name'], values['parameters'])
                    df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
                    concat_filepath = os.path.join(arglist['SAVE'],'%s_%s_HEI.csv'%(option, name))
                    df=df.drop_duplicates(subset=['Visit_label', 'Participant ID','Date of Intake','Date_taken'],keep='first')
                    df.to_csv(concat_filepath, index=False, sep=",", header=True)
                else:
                    toSum=['HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEIX2_TOTALFRUIT'  ,
                           'HEIX5_PROTEIN' ,  'HEIX7_FRUITJUICE' , 'HEIX8_SSB', 'HEIX9_SWEETS',
                           'HEIX10_SALTY', 'HEIX11_CEREAL']
                    infant_DQI(df,key, values['name'], values['parameters'])
                    df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
                    concat_filepath = os.path.join(arglist['SAVE'],'%s_%s_HEI.csv'%(option, name))
                    df=df.drop_duplicates(subset=['Visit_label', 'Participant ID','Date of Intake','Date_taken'],keep='first')
                    df.to_csv(concat_filepath, index=False, sep=",", header=True)

In [186]:
for key, value in split_dict.items():
    if key == 'DF_child':
        DQI_BF(value, 'HEIX0_BREASTFEEDING', 'child')
        check(ped_dict['child'], value, 'child', key, arglist)
    elif key == 'DF_young':
        DQI_BF(value, 'HEIX0_BREASTFEEDING', 'young')
        check(ped_dict['young'], value, 'young', key, arglist)
    else:
        DQI_BF(value, 'HEIX0_BREASTFEEDING', 'infant')
        check(ped_dict['infant'], value, 'infant', key, arglist)

Calculating score for hei_vegetables
now calculating HEIX1_VEGETABLES
Calculating score for hei_totfruit
now calculating HEIX2_TOTALFRUIT
Calculating score for hei_wholegrains
now calculating HEIX3_WHOLEGRAIN
Calculating score for hei_milk
now calculating HEIX4_TOTALDAIRY
Calculating score for hei_proteins
now calculating HEIX5_PROTEIN
Calculating score for hei_refinedgrains
now calculating HEIX6_REFINEDGRAIN
Calculating score for hei_fruitjuice
now calculating HEIX7_FRUITJUICE
Calculating score for hei_SSB
now calculating HEIX8_SSB
Calculating score for hei_sweets
now calculating HEIX9_SWEETS
Calculating score for hei_salty
now calculating HEIX10_SALTY
Calculating score for hei_vegetables
now calculating HEIX1_VEGETABLES
Calculating score for hei_totfruit
now calculating HEIX2_TOTALFRUIT
Calculating score for hei_wholegrains
now calculating HEIX3_WHOLEGRAIN
Calculating score for hei_milk
now calculating HEIX4_TOTALDAIRY
Calculating score for hei_proteins
now calculating HEIX5_PROTEIN


In [182]:
split_dict['DF_young']['HEI2015_TOTAL_SCORE'].describe()

count    237.000000
mean      39.396624
std        6.485550
min       25.000000
25%       35.000000
50%       39.000000
75%       42.500000
max       55.000000
Name: HEI2015_TOTAL_SCORE, dtype: float64

In [179]:
split_dict['DF_infant']['HEI2015_TOTAL_SCORE'].describe()

count    364.000000
mean      36.263736
std        5.608603
min       20.000000
25%       30.000000
50%       40.000000
75%       40.000000
max       40.000000
Name: HEI2015_TOTAL_SCORE, dtype: float64

In [181]:
split_dict['DF_infant'].shape

(364, 496)